In [4]:
import sys
!{sys.executable} -m pip install pandas oxrdflib Pygments

import pandas as pd
from IPython.display import display, HTML
from pygments import highlight
from pygments.lexers import SparqlLexer
from pygments.formatters import HtmlFormatter
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery

def run_query(graph, query_path):
    try:
        with open(query_path, 'r') as file:
            query = file.read()
    except Exception as _e:
        print(f"No file for {query_path}")
        return
    results = graph.query(query)
    # Display the SPARQL query
    formatted_query = highlight(query, SparqlLexer(), HtmlFormatter(style='solarized-dark', full=True, nobackground=True))
    display(HTML(formatted_query))
    # Convert results to a Pandas DataFrame
    res_list = []
    for row in results:
        res_list.append([str(item) for item in row])
    df = pd.DataFrame(res_list, columns=[str(var) for var in results.vars]) if len(res_list) > 0 else pd.DataFrame()
    # Display the DataFrame as a table in Jupyter Notebook
    display(HTML(df.to_html()))

g = Graph(store="Oxigraph")

g.parse("output/graph/models-metrics-papers.ttl", format="ttl")
print(len(g))

4149107


## **QUERIES TO ASSESS THE QUALITY OF THE GRAPH**

##### Number of instances for each class

In [3]:
run_query(g, "queries/quality/q1.rq")

,class,count
0,Paper,342
1,HyperParameter,14
2,EvaluationMetric,666427
3,Model,18683


##### Identify models missing names

In [4]:
run_query(g, "queries/quality/q2.rq")

""


##### Models missing hyperparameters

In [4]:
run_query(g, "queries/quality/q3.rq")

,class,count
0,Model,18669


##### Domain inconsistencies: this query will return all triples where there is a potential domain inconsistency - e.g. the property declares a domain but the subject is not typed as an instance of that domain.

In [10]:
run_query(g, "queries/quality/q4.rq")

""


#### Range inconsistencies

In [10]:
# This helped me to understand that I used a wrong range for the "creator" property. I had used a Literal instead of a URI.
# I fixed this in the data generation script and re-generated the data.

run_query(g, "queries/quality/q5.rq")

""


#### Multiple models with the same name

In [11]:
run_query(g, "queries/quality/q6.rq")

""


#### Untyped entities

In [ ]:
run_query(g, "queries/quality/q7.rq")

# Sincere this returned http://schema.org/Person because I didn't assert that Person is a class in my graph, in order to ensure consistency
# but also because I am not importing the full schema ontology I decided to create the Person class in my graph.

,instance
0,http://schema.org/Person


#### Duplicated triples

In [8]:
run_query(g, "queries/quality/q8.rq")

""


# **QUERIES TO GAIN INSIGHTS FROM THE GRAPH**

#### Top 5 most downloaded models for each task

In [25]:
run_query(g, "queries/results/q1.rq")

,name,task,downloads
0,ast-finetuned-speech-commands-v2,audio-classification,54486
1,diar_sortformer_4spk-v1,audio-classification,5350
2,distil-ast-audioset-finetuned-cry,audio-classification,1002
3,wav2vec2_turkish_gender_classification,audio-classification,402
4,genre-recognizer-finetuned-gtzan_dset,audio-classification,383
5,sepformer-dns4-16k-enhancement,audio-to-audio,26265
6,open-universe,audio-to-audio,17603
7,EBEN_temple_vibration_pickup,audio-to-audio,13375
8,EBEN_throat_microphone,audio-to-audio,146
9,EBEN_rigid_in_ear_microphone,audio-to-audio,107


#### Given a task, look for the most used datasets. The query shows image-classification models

In [9]:
run_query(g, "queries/results/q2.rq")

,dataset,metricCount
0,Unknown,573
1,glue,546
2,emotion,489
3,imdb,346
4,GLUE QQP,254
5,GLUE MRPC,221
6,tweet_eval,217
7,GLUE STSB,123
8,GLUE MNLI,111
9,GLUE QNLI,111


#### By finetuning a specific model, (e.g., BERT, gpt2, xlm-roberta-large etc..) for which tasks can I use it?

In [39]:
run_query(g, "queries/results/q3.rq")

,task
0,text-classification
1,token-classification
2,sentence-similarity


### All the text-classification models (or any other task) that supports more than one language (or a specific one)

In [48]:
# Query for more than one language
run_query(g, "queries/results/q4.rq")

# Query for a specific language (e.g., en, it, fr ...)
run_query(g, "queries/results/q5.rq")

,name,languageCount
0,ModernBERT-base-long-context-qe-v1,26
1,ModernBERT-large-qe-v1,26
2,multilingual_minilm-amazon_massive-intent_eu7,7
3,zephyr-dpo-v2,2
4,BAAI-bge-reranker-large,2
5,pixel-base-finetuned-xnli-translate-train-all,15
6,xlm-roberta-base-sentiment-multilingual-finetuned,3
7,ModernBERT-large-qe-maxlen512-v1,26
8,ModernBERT-base-qe-v1,26
9,ModernBERT-base-qe-maxlen512-lr3e-04-v1,26


,name
0,Jajuka-3b
1,Llama-3.1-8B-AlpaCare-MedInstruct-GGUF
2,ultiima-72B
3,openbuddy-nemotron-70b-v23.1-131k
4,openbuddy-deepseek-67b-v18.1-4k


#### What are the most common metrics when evaluating a model on a given task?

In [12]:
run_query(g, "queries/results/q6.rq")

,task,metricType,metricCount
0,token-classification,accuracy,7793
1,token-classification,f1,1319
2,token-classification,recall,1212
3,token-classification,precision,1198
4,token-classification,f1_macro,128
5,token-classification,precision_entity_span,128
6,token-classification,recall_macro,128
7,token-classification,precision_macro,128
8,token-classification,recall_entity_span,128
9,token-classification,f1_entity_span,128


### Given a dataset and a task, which model achieves the highest accuracy, f1, recall, and precision?

In [10]:
run_query(g, "queries/results/q7.rq")

,name,dataset,metricValue,metricType
0,bert-base-uncased-sst2-epochs-2-lr-0.0001,glue,0.99,accuracy
1,deberta-v3-small-finetuned-sst2,glue,0.94170403,f1
2,deberta-v3-small-finetuned-sst2,glue,0.9375,precision
3,bert-base-uncased-mrpc,glue,0.9641577,recall


#### Retrieve models with the hyperparameters used at training time

In [3]:
run_query(g, "queries/results/q9.rq")

,modelName,hpName,hpValue
0,DeBERTa-v3-base-mnli-fever-anli,fp16,true
1,DeBERTa-v3-base-mnli-fever-anli,learning_rate,2e-05
2,DeBERTa-v3-base-mnli-fever-anli,num_train_epochs,3
3,DeBERTa-v3-base-mnli-fever-anli,per_device_eval_batch_size,32
4,DeBERTa-v3-base-mnli-fever-anli,per_device_train_batch_size,32
5,DeBERTa-v3-base-mnli-fever-anli,warmup_ratio,0.1
6,DeBERTa-v3-base-mnli-fever-anli,weight_decay,0.06
7,amd-partial-v1,batch_size,"[64, 64]"
8,amd-partial-v1,body_learning_rate,"[2e-05, 1e-05]"
9,amd-partial-v1,distance_metric,cosine_distance


#### Retrieve models along with papers, abstract and disadvantages and advantages extract from the article

In [5]:
run_query(g, "queries/results/q10.rq")

,modelName,paperID,paperTitle,paperSummary,advantages,limitations
0,financial-summarization-pegasus,1912.08777,PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive\n Summarization,"Recent work pre-training Transformers with self-supervised objectives on\nlarge text corpora has shown great success when fine-tuned on downstream NLP\ntasks including text summarization. However, pre-training objectives tailored\nfor abstractive text summarization have not been explored. Furthermore there is\na lack of systematic evaluation across diverse domains. In this work, we\npropose pre-training large Transformer-based encoder-decoder models on massive\ntext corpora with a new self-supervised objective. In PEGASUS, important\nsentences are removed/masked from an input document and are generated together\nas one output sequence from the remaining sentences, similar to an extractive\nsummary. We evaluated our best PEGASUS model on 12 downstream summarization\ntasks spanning news, science, stories, instructions, emails, patents, and\nlegislative bills. Experiments demonstrate it achieves state-of-the-art\nperformance on all 12 downstream datasets measured by ROUGE scores. Our model\nalso shows surprising performance on low-resource summarization, surpassing\nprevious state-of-the-art results on 6 datasets with only 1000 examples.\nFinally we validated our results using human evaluation and show that our model\nsummaries achieve human performance on multiple datasets.",Achieves state-of-the-art performance on all 12 downstream datasets measured by ROUGE scores. Surpasses previous state-of-the-art results on 6 datasets with only 1000 examples. Summaries achieve human performance on multiple datasets.,There has been little work on systematic evaluation of models across diverse domains.
1,jina-embeddings-v3,2409.10173,jina-embeddings-v3: Multilingual Embeddings With Task LoRA,"We introduce jina-embeddings-v3, a novel text embedding model with 570\nmillion parameters, achieves state-of-the-art performance on multilingual data\nand long-context retrieval tasks, supporting context lengths of up to 8192\ntokens. The model includes a set of task-specific Low-Rank Adaptation (LoRA)\nadapters to generate high-quality embeddings for query-document retrieval,\nclustering, classification, and text matching. Additionally, Matryoshka\nRepresentation Learning is integrated into the training process, allowing\nflexible truncation of embedding dimensions without compromising performance.\nEvaluation on the MTEB benchmark shows that jina-embeddings-v3 outperforms the\nlatest proprietary embeddings from OpenAI and Cohere on English tasks, while\nachieving superior performance compared to multilingual-e5-large-instruct\nacross all multilingual tasks.","Achieves state-of-the-art performance on multilingual data and long-context retrieval tasks, supporting context lengths of up to 8192 tokens. Includes task-specific Low-Rank Adaptation (LoRA) adapters for high-quality embeddings. Integrates Matryoshka Representation Learning for flexible truncation of embedding dimensions without compromising performance. Outperforms latest proprietary embeddings from OpenAI and Cohere on English tasks and multilingual-e5-large-instruct on all multilingual tasks. More cost-efficient compared to LLM-based embeddings like e5-mistral-7b-instruct.",Traditional embedding models often require fine-tuning for specific tasks and struggle with common failure cases. Large language models (LLMs) as the backbone for general-purpose embedding generation pose challenges in real-world applications and offer marginal improvements compared to encoder-only embedding models.
2,kotoba-whisper-v2.0,2212.04356,Robust Speech Recognition via Large-Scale Weak Supervision,"We study the capabilities of speech processing systems trained simply to\npredict large amounts of transcripts of audio on the internet. When scaled to\n680,000 hours of multilingual and multitask supervision, the resulting mod

In [6]:
run_query(g, "queries/results/q11.rq")